<a href="https://colab.research.google.com/github/JAIDHEER007/Jhon-Conway-Game-of-Life/blob/main/Python%20Implementation/GOL_JSON_INFO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from PIL import Image
from time import perf_counter
import os
from os.path import join
import datetime
import cv2
import re
import shutil
import json

In [2]:
def createFolder(generations, fps, grid) -> str:
    # Current Working Directory
    pdir = os.getcwd()
    
    #Getting Current Time
    curr = datetime.datetime.now()
    dateStr = str(curr.date())
    timeStr = str(curr.time())

    #Folder name
    directory = "GOL_RUN_" + dateStr.replace('-','_') + "_" + timeStr.replace(':', '_')
    
    #Final Path
    fpath = join(pdir, directory)
    
    #Creating the directory
    os.mkdir(fpath)
    
    # Creating the Images SubFolder
    imgPath = join(fpath, 'Images')
    os.mkdir(imgPath)
    
    # Writing usefull files
    
    # Saving the inital grid state
    np.savetxt(fname = join(fpath, 'initalState.csv'), X = grid, delimiter = ',', fmt = '%d')
    
    info = {
        "author": "S. Jaidheer", 
        "github": "JAIDHEER007", 
        "date": None, 
        "time": None, 
        "data": {
            "game": {
                "rows": 0, 
                "cols": 0, 
                "generations": 0, 
                "fps": 0
            }, 
            "file": {}
        }
    }

    info["date"] = dateStr
    info["time"] = timeStr
    info["data"]["game"]["rows"] = grid.shape[0]
    info["data"]["game"]["cols"] = grid.shape[0]
    info["data"]["game"]["generations"] = generations
    info["data"]["game"]["fps"] = fps
           
    with open(join(fpath, 'info.json'), 'w') as fileHandle:
        json.dump(info, fileHandle)
           
    return fpath

In [3]:
class gol:
    def isValid(i:int, j:int, rows:int, cols:int) -> bool:
        return (i >= 0) and (i < rows) and (j >= 0) and (j < cols)
    
    def rules(lives:int, currState:bool) -> bool:
        if currState:
            if lives > 3 or lives < 2:
                return False
            else: return True  
        return (lives == 3)
    
    def generationUpdate(grid):
        rows, cols = grid.shape
        # print(rows, cols)
        nextGeneration = np.zeros((rows, cols), dtype = bool)
        for i in range(rows):
            for j in range(cols):

                # Finding the Live Neighbours
                lives = 0
                for m in [-1, 0, 1]:
                    for n in [-1, 0, 1]:
                        if m == 0 and n == 0: continue
                        lives += int(grid[(((i + m) + rows) % rows), (((j + n) + cols) % cols)]);
                            
                nextGeneration[i, j] = gol.rules(lives, grid[i, j])

        return nextGeneration
        
    def __init__(self, startGrid, generations, fps = 35):
        self.__rows, self.__cols = startGrid.shape 
        self.__generations = generations
        self.__fps = fps
        self.__grid = startGrid

        #Folder Path
        self.__fpath = createFolder(generations = self.__generations, 
                                    fps = self.__fps, 
                                    grid = self.__grid)
    
    def start(self):
        if self.__rows > 200 or self.__cols > 200:
            raise Exception("Grid Limit Reached")
        
        imgPath = join(self.__fpath, 'Images')
        for generation in range(self.__generations):
            img = Image.fromarray(self.__grid)
            img = img.resize((self.__rows * 10, self.__cols * 10))
            img.save(join(imgPath, 'Img{generationCount}.png'.format(generationCount = generation)))
            self.__grid = gol.generationUpdate(self.__grid)
        
    def saveVideo(self):
        imgPath = join(self.__fpath, 'Images')

        if not os.path.exists(imgPath):
            raise Exception("Call start Method to create images")
        
        video_name = join(self.__fpath, "Output.mp4")

        images = [img for img in os.listdir(imgPath) if img.endswith(".png")]

        if len(images) == 0: 
            raise Exception("No Images were created. Call start() method")

        pattern = re.compile(r'[0-9]+')
        images.sort(key = lambda image: int(pattern.findall(image)[-1]))

        frame = cv2.imread(os.path.join(imgPath, images[0]))

        height, width = frame.shape[:2]

        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        video = cv2.VideoWriter(video_name, fourcc, 35, (width, height)) 

        for image in images:
            video.write(cv2.imread(os.path.join(imgPath, image)))

        cv2.destroyAllWindows() 
        video.release()

    def deleteFolder(self):
        shutil.rmtree(self.__fpath)


In [4]:
if __name__ == '__main__':
    rows = 175
    cols = 175
    
    grid = np.random.choice(a = [True, False], size = (rows, cols))
    
    gol1 = gol(startGrid = grid, generations = 100)

In [5]:
gol1.start()
gol1.saveVideo()